# **Calculate Course Similarity using BoW Features**


In [1]:
# uncomment the below imports if you haven't done it before
# !pip install nltk==3.6.7
# !pip install gensim==4.1.2

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import pandas as pd
import nltk as nltk

from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams
from gensim import corpora

%matplotlib inline

In [3]:
# also set a random state
rs = 123

### Calculate the consine similarity between two example courses


Suppose we have two simple example courses:


In [4]:
course1 = "machine learning for everyone"

In [5]:
course2 = "machine learning for beginners"

Next we can quickly tokenize them using the split() method (or using `word_tokenize()` method provided in `nltk` as we did in the previous lab).


In [6]:
tokens = set(course1.split() + course2.split())

In [7]:
tokens = list(tokens)
tokens

['learning', 'for', 'everyone', 'beginners', 'machine']

then generate BoW features (token counts) for these two courses (or using `tokens_dict.doc2bow()` method provided in `nltk`, similar to what we did in the previous lab).


In [8]:
def generate_sparse_bow(course):
    bow_vector = []
    words = course.split()
    for token in tokens:
        if token in words:
            bow_vector.append(1)
        else:
            bow_vector.append(0)
    return bow_vector

In [9]:
bow1 = generate_sparse_bow(course1)
bow1

[1, 1, 1, 0, 1]

In [10]:
bow2 = generate_sparse_bow(course2)
bow2

[1, 1, 0, 1, 1]

From the above cell outputs, we can see the two vectors are very similar. Only two dimensions are different.


Now we can quickly apply the cosine similarity measurement on the two vectors:


In [11]:
cos_sim = 1 - cosine(bow1, bow2)

In [12]:
print(f"The cosine similarity between course '{course1}' and course \
'{course2}' is {round(cos_sim, 2) * 100}%")

The cosine similarity between course 'machine learning for everyone' and course 'machine learning for beginners' is 75.0%


_Practice: Try other similarity measurements such as Euclidean Distance or Jaccard index._


In [13]:
# WRITE YOUR CODE HERE

import math
print(f"MATH MODULE:\nThe Euclidean distance is {math.dist(bow1, bow2)}\n")

from scipy.spatial import distance
print(f"SCIPY MODULE:\nThe Euclidean distance is {distance.euclidean(bow1, bow2)}\n")

def eucl(l1, l2):
    z = []
    for i in range(len(l1)):
        z.append((l1[i]-l2[i])**2)
    return sum(z)**0.5

print(f"BY FORMULA:\nThe Euclidean distance is {eucl(bow1, bow2)}\n")


MATH MODULE:
The Euclidean distance is 1.4142135623730951

SCIPY MODULE:
The Euclidean distance is 1.4142135623730951

BY FORMULA:
The Euclidean distance is 1.4142135623730951



For Example: Euclidean distance between 2 points $p$ and $q$ can be summarized by this equation: $d(p,q)={\sqrt {(p_{1}-q_{1})^{2}+(p_{2}-q_{2})^{2}+(p_{3}-q_{3})^{2}}}$. You can use `euclidean(p,q)` function from ```scipy``` package to calculate it. 


>The Jaccard similarity index measures the similarity between two sets of data. It can range from 0 to 1. The higher the number, the more similar the two sets of data.

In [14]:
print(course1.split(), course2.split(), sep='\n')

['machine', 'learning', 'for', 'everyone']
['machine', 'learning', 'for', 'beginners']


In [15]:
# Jaccard index:

def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

print(f"BY FORMULA:\nThe Jaccard index {jaccard(course1.split(), course2.split())}")


BY FORMULA:
The Jaccard index 0.6


### TASK: Find similar courses to the course `Machine Learning with Python`


Now you have learned how to calculate cosine similarity between two sample BoW feature vectors. Let's work on some real course BoW feature vectors.


In [16]:
# Load the BoW features as Pandas dataframe
bows_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/courses_bows.csv"
bows_df = pd.read_csv(bows_url)
bows_df = bows_df[['doc_id', 'token', 'bow']]

In [17]:
bows_df.head(10)

,doc_id,token,bow
0,ML0201EN,ai,2
1,ML0201EN,apps,2
2,ML0201EN,build,2
3,ML0201EN,cloud,1
4,ML0201EN,coming,1
5,ML0201EN,create,1
6,ML0201EN,data,1
7,ML0201EN,developer,1
8,ML0201EN,found,1
9,ML0201EN,fun,1


The `bows_df` dataframe contains the BoW features vectors for each course, in a vertical and dense format. It has three columns `doc_id` represents the course id, `token` represents the token value, and `bow` represents the BoW value (token count).


Then, let's load another course content dataset which contains the course title and description:


In [18]:
# Load the course dataframe
course_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/course_processed.csv"
course_df = pd.read_csv(course_url)

In [19]:
course_df.head(10)

,COURSE_ID,TITLE,DESCRIPTION
0,ML0201EN,robots are coming build iot apps with watson ...,have fun with iot and learn along the way if ...
1,ML0122EN,accelerating deep learning with gpu,training complex deep learning models with lar...
2,GPXX0ZG0EN,consuming restful services using the reactive ...,learn how to use a reactive jax rs client to a...
3,RP0105EN,analyzing big data in r using apache spark,apache spark is a popular cluster computing fr...
4,GPXX0Z2PEN,containerizing packaging and running a sprin...,learn how to containerize package and run a ...
5,CNSC02EN,cloud native security conference data security,introduction to data security on cloud
6,DX0106EN,data science bootcamp with r for university pr...,a multi day intensive in person data science ...
7,GPXX0FTCEN,learn how to use docker containers for iterati...,learn how to use docker containers for iterati...
8,RAVSCTEST1,scorm test 1,scron test course
9,GPXX06RFEN,create your first mongodb database,in this guided project you will get started w...


Given course ID `ML0101ENv3`, let's find out its title and description:


In [20]:
course_df[course_df['COURSE_ID'] == 'ML0101ENv3']

,COURSE_ID,TITLE,DESCRIPTION
158,ML0101ENv3,machine learning with python,machine learning can be an incredibly benefici...


We can see it is a machine learning with Python course so we can expect any machine learning or Python related courses would be similar.


Then, let's print its associated BoW features:


In [21]:
ml_course = bows_df[bows_df['doc_id'] == 'ML0101ENv3']
ml_course

,doc_id,token,bow
2747,ML0101ENv3,course,1
2748,ML0101ENv3,learning,4
2749,ML0101ENv3,machine,3
2750,ML0101ENv3,need,1
2751,ML0101ENv3,get,1
2752,ML0101ENv3,started,1
2753,ML0101ENv3,python,2
2754,ML0101ENv3,tool,1
2755,ML0101ENv3,tools,1
2756,ML0101ENv3,predict,1


We can see the BoW feature vector is in vertical format but normally feature vectors are in horizontal format. One way to transpose the feature vector from vertical to horizontal is to use the Pandas `pivot()` method:


In [22]:
ml_courseT = ml_course.pivot(index=['doc_id'], columns='token').reset_index(level=[0])
ml_courseT

doc_id        bow                                              \
token             beneficial course free future get give hidden insights   
0      ML0101ENv3          1      1    1      1   1    1      1        1   

                                                                          \
token learning machine need predict python started supervised tool tools   
0            4       3    1       1      2       1          1    1     1   

                           
token trends unsupervised  
0          1            1

To compare the BoWs of any two courses, which normally have a different set of tokens, we need to create a union token set and then transpose them. We have provided a method called `pivot_two_bows` as follows:


In [23]:
def pivot_two_bows(basedoc, comparedoc):
    base = basedoc.copy()
    base['type'] = 'base'
    compare = comparedoc.copy()
    compare['type'] = 'compare'
    # Append the two token sets vertically
    join = base.append(compare)
    # Pivot the two joined courses
    joinT = join.pivot(index=['doc_id', 'type'], columns='token').fillna(0).reset_index(level=[0, 1])
    # Assign columns
    joinT.columns = ['doc_id', 'type'] + [t[1] for t in joinT.columns][2:]
    return joinT

In [24]:
course1 = bows_df[bows_df['doc_id'] == 'ML0151EN']
course2 = bows_df[bows_df['doc_id'] == 'ML0101ENv3']

In [25]:
import warnings
warnings.filterwarnings('ignore')

bow_vectors = pivot_two_bows(course1, course2)
bow_vectors

,doc_id,type,approachable,basics,beneficial,comparison,course,dives,free,future,...,relates,started,statistical,supervised,tool,tools,trends,unsupervised,using,vs
0,ML0101ENv3,compare,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,ML0151EN,base,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


Similarly, we can use the cosine method to calculate their similarity:


In [26]:
similarity = 1 - cosine(bow_vectors.iloc[0, 2:], bow_vectors.iloc[1, 2:])
similarity

0.6626221399549089

Now it's your turn to perform a task of finding all courses similar to the course `Machine Learning with Python`:


In [27]:
course_df[course_df['COURSE_ID'] == 'ML0101ENv3']

,COURSE_ID,TITLE,DESCRIPTION
158,ML0101ENv3,machine learning with python,machine learning can be an incredibly benefici...


You can set a similarity threshold such as 0.5 to determine if two courses are similar enough.


_TODO: Find courses which are similar to course `Machine Learning with Python (ML0101ENv3)`, you also need to show the title and descriptions of those courses._


In [28]:
# first a for loop to merge the two rows
# then comes the condition if similarity > 0.5..

unqiue_ids = sorted(list(set(x for x in course_df['COURSE_ID'] if x != 'ML0101ENv3')))
similar_to = []
for x in unqiue_ids:
    course1 = bows_df[bows_df['doc_id'] == 'ML0151EN']
    course2 = bows_df[bows_df['doc_id'] == x]
    bow_vectors = pivot_two_bows(course1, course2)
    
    similarity = 1 - cosine(bow_vectors.iloc[0, 2:], bow_vectors.iloc[1, 2:])
    
    if similarity > 0.5:
        similar_to.append(x)

In [29]:
sim_courses = []
for x in similar_to:
    ind = np.where(course_df["COURSE_ID"] == x)
    a = ind[0][0]
    sim_courses.append(a)

print('The following courses are similar to "Machine Learning with Python (ML0101ENv3)"\n')
print(*similar_to, sep=' | ') 
print()
print("------------------------ Details ------------------------")
for i, elem in enumerate(similar_to):
    print(f'Course_ID:\t {elem:<15} Title:\t {course_df.iloc[sim_courses[i], 1]}')


The following courses are similar to "Machine Learning with Python (ML0101ENv3)"

ML0109EN | ML0151EN | excourse46 | excourse47 | excourse51 | excourse60 | excourse69

------------------------ Details ------------------------
Course_ID:	 ML0109EN        Title:	 machine learning   dimensionality reduction
Course_ID:	 ML0151EN        Title:	 machine learning with r
Course_ID:	 excourse46      Title:	 machine learning
Course_ID:	 excourse47      Title:	 machine learning for all
Course_ID:	 excourse51      Title:	 introduction to machine learning in production
Course_ID:	 excourse60      Title:	 introduction to tensorflow for artificial intelligence  machine learning  and deep learning
Course_ID:	 excourse69      Title:	 machine learning with big data
